In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display

In [3]:
df_food_scaled = pd.read_csv('df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_7096\4074791732.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('df_food_scaled.csv')


In [4]:
nutrient_info = {
    'folate_eaten':                                         {'unit': 'mcg', 'target': 400,  'type': 'qualifying'},
    'niacin_eaten':                                         {'unit': 'mg',  'target': 14,   'type': 'qualifying'},
    'pantothenic_acid_eaten':                               {'unit': 'mg',  'target': 5,    'type': 'qualifying'},
    'vitamin_b2_eaten':                                     {'unit': 'mg',  'target': 1.1,  'type': 'qualifying'},
    'vitamin_b1_eaten':                                     {'unit': 'mg',  'target': 1.2,  'type': 'qualifying'},
    'all_trans_retinol_equivalents_activity_eaten':         {'unit': 'IU',  'target': 700,  'type': 'qualifying'},
    'beta_carotene_eaten':                                  {'unit' : 'mcg','target' : 700, 'type' :'qualifying'},
    'vitamin_b12_eaten':                                    {'unit': 'mcg', 'target': 2.4,  'type': 'qualifying'},
    'vitamin_b6_eaten':                                     {'unit': 'mg',  'target': 1.3,  'type': 'qualifying'},
    'vitamin_c_eaten':                                      {'unit': 'mg',  'target': 75,   'type': 'qualifying'},
    'vitamin_d_eaten':                                      {'unit': 'mcg', 'target': 7.5,  'type': 'qualifying'},
    'vitamin_e_activity_eaten':                             {'unit': 'TAE', 'target': 15,    'type': 'qualifying'},
    'calcium_eaten':                                        {'unit': 'mg',  'target': 1000, 'type': 'qualifying'},
    'iron_eaten':                                           {'unit': 'mg',  'target': 18,   'type': 'qualifying'},
    'magnesium_eaten':                                      {'unit': 'mg',  'target': 320,  'type': 'qualifying'},
    'phosphorus_eaten':                                     {'unit': 'mg',  'target': 700,  'type': 'qualifying'},
    'potassium_eaten':                                      {'unit': 'mg',  'target': 4700, 'type': 'qualifying'},
    'zinc_eaten':                                           {'unit': 'mg',  'target': 8,    'type': 'qualifying'},
    'fiber_eaten':                                          {'unit': 'g',   'target': 25,   'type': 'qualifying'},
    'protein_eaten':                                        {'unit': 'g',   'target': 46,   'type': 'qualifying'},
    'fat_eaten':                                            {'unit': 'g',   'target': 78,   'type': 'disqualifying'},
    'fatty_acids_saturated_eaten':                          {'unit': 'g',   'target': 22,   'type': 'disqualifying'},
    'cholesterol_eaten':                                    {'unit': 'mg',  'target': 300,  'type': 'disqualifying'},
    'sugar_eaten':                                          {'unit': 'g',   'target': 125,  'type': 'disqualifying'},
    'sodium_eaten':                                         {'unit': 'mg',  'target': 2400, 'type': 'disqualifying'},
    'salt_eaten' :                                          {'unit' : 'g',  'target': 6,   'type': 'disqualifying'}
}


conversion_factors = {
    "mg": 1000,
    "g": 1,
    "mcg": 1000000,
    "IU": 1,
    "TAE": 1,
}

In [5]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [6]:
df_butter_margarine_spreads = df_food_scaled[df_food_scaled['food_group'] == 'butter_margarine_spreads']
df_butter_margarine_spreads.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
290425,Délice : extra Light,3.777249,1.079267,52.906567
301075,le Parfait Wurziger Brotaufstrich,3.147747,2.552177,71.926246
305905,Becel: Pro Activ,2.592644,0.929681,45.535122
172217,Margarine,2.445222,0.994892,45.603756
284495,Coop Qualité & Prix - Margarine,1.305216,1.314421,40.631510
...,...,...,...,...
161878,Vegetable fat,0.000000,0.748403,0.000000
298635,aha - KOKOS Margarine,0.000000,1.277303,0.000000
297998,Natur Kraft Werke - Kürbiskernmus,0.000000,0.749318,0.000000
311133,Coop Naturaplan - Bio Butter,0.000000,0.360202,0.000000


In [7]:
high_qi_butter_margarine_spreads = df_butter_margarine_spreads[df_butter_margarine_spreads['QI'] > 1].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [8]:
high_qi_butter_margarine_spreads[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
290425,Délice : extra Light,5.0,3.777249,1.079267,52.906567
301075,le Parfait Wurziger Brotaufstrich,25.0,3.147747,2.552177,71.926246
305905,Becel: Pro Activ,10.0,2.592644,0.929681,45.535122
172217,Margarine,5.0,2.445222,0.994892,45.603756
284495,Coop Qualité & Prix - Margarine,10.0,1.305216,1.314421,40.631510
284129,Volg - Margarine,10.0,1.270157,0.516791,40.688617
84402,Sesame paste,10.0,1.223884,0.601614,61.104151
277746,MIGROS Leger FIT-ONNAISE CLASSIC,7.0,1.188645,1.517040,28.165576
277730,Nestlé THOMY Thomynaise light 60% weniger Fett,20.0,1.117862,1.695834,26.322371
284854,M Budget - Margarine,10.0,1.063092,1.361600,38.315429


In [9]:
ratio_cols = [i for i in high_qi_butter_margarine_spreads.columns if '_ratio_scaled' in i]

df_corr = high_qi_butter_margarine_spreads[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

calcium_eaten_ratio_scaled                                  -0.230954
magnesium_eaten_ratio_scaled                                -0.223934
vitamin_e_activity_eaten_ratio_scaled                       -0.089371
cholesterol_eaten_ratio_scaled                              -0.066013
phosphorus_eaten_ratio_scaled                               -0.003088
protein_eaten_ratio_scaled                                   0.059167
fat_eaten_ratio_scaled                                       0.063165
potassium_eaten_ratio_scaled                                 0.083161
salt_eaten_ratio_scaled                                      0.094208
iron_eaten_ratio_scaled                                      0.137926
zinc_eaten_ratio_scaled                                      0.222515
fatty_acids_saturated_eaten_ratio_scaled                     0.239290
niacin_eaten_ratio_scaled                                    0.263989
fiber_eaten_ratio_scaled                                     0.317678
sodium_eaten_ratio_s

In [10]:
def compute_index(row, nutrient_cols, scaling_factor=2000) :

    index = 0
    ratio_sum = 0

    for nutr in nutrient_cols :
        ratio_sum += row[nutr + '_ratio']

    index = (scaling_factor / row['energy_kcal_eaten']) * (ratio_sum / len(nutrient_cols))
    return index

In [11]:
def compute_qi_excluding(row, nutrient_list, exclude=None, scaling_factor=2000):
    if exclude is not None:
        new_list = [nutr for nutr in nutrient_list if nutr != exclude]
    else:
        new_list = nutrient_list
    

    return compute_index(row, new_list, scaling_factor=scaling_factor)

In [12]:
def compute_qi_excluding_multiple(row, nutrient_list, exclude_list, scaling_factor=2000):

    new_list = nutrient_list.copy()  

    for nutr in exclude_list:
        if nutr in new_list:
            new_list.remove(nutr)

    return compute_index(row, new_list, scaling_factor=scaling_factor)

In [13]:
def compare_qi_excluding_nutrient(df, nutrient_to_exclude, qualifying_nutrients, new_col_name=None,scaling_factor=2000):


    if new_col_name is None:
        new_col_name = f"QI_excl_{nutrient_to_exclude}"


    df[new_col_name] = df.apply(lambda row: compute_qi_excluding(row, qualifying_nutrients, exclude=nutrient_to_exclude, scaling_factor=scaling_factor), axis=1)

 
    df_plot = df.drop_duplicates('combined_name').copy()
    df_plot = df_plot[['combined_name', 'QI', new_col_name]]

    labels = df_plot['combined_name'].tolist()
    x = np.arange(len(labels))
    width = 0.35


    plt.figure(figsize=(16, 8))
    plt.bar(x - width/2, df_plot['QI'], width, label='QI (incl. ' + nutrient_to_exclude + ')', color='skyblue')
    plt.bar(x + width/2, df_plot[new_col_name], width, label=f"QI (excl. {nutrient_to_exclude})", color='deeppink')

    plt.xlabel('Food Item')
    plt.ylabel('QI Value')
    plt.title(f"Comparison of QI with and without {nutrient_to_exclude}")
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()

    df_plot['QI_diff'] = df_plot['QI'] - df_plot[new_col_name]
    df_plot['QI_pct_change'] = (df_plot['QI_diff'] / df_plot['QI']) * 100

    return df_plot

In [14]:
def compute_nb(row, nutrient_cols, scaling_factor=2000) :
    truncated_ratios = []
    for nutr in nutrient_cols:
        ratio =   row[nutr + '_ratio_scaled']
        if ratio > 1.0 :
            ratio = 1
        truncated_ratios.append(ratio)

    nb_value =  (sum(truncated_ratios) / len(nutrient_cols)) * 100
    return nb_value

In [15]:
def compute_nb_excluding(row, nutrient_list, exclude=None, scaling_factor=2000):

    if exclude is not None:
        new_list = [nutr for nutr in nutrient_list if nutr != exclude]
    else:
        new_list = nutrient_list
    return compute_nb(row, new_list, scaling_factor=scaling_factor)


In [16]:
def compare_nb_excluding_nutrient(df, nutrient_to_exclude, qualifying_nutrients, new_col_name=None, scaling_factor=2000):
  
    if new_col_name is None:
        new_col_name = f"NB_excl_{nutrient_to_exclude}"

    df['NB'] = df.apply(lambda row: compute_nb(row, qualifying_nutrients, scaling_factor=scaling_factor), axis=1)

    df[new_col_name] = df.apply(lambda row: compute_nb_excluding(row, qualifying_nutrients, exclude=nutrient_to_exclude, scaling_factor=scaling_factor), axis=1)
    
    df_plot = df.drop_duplicates('combined_name').copy()
    df_plot = df_plot[['combined_name', 'NB', new_col_name]]
    
    labels = df_plot['combined_name'].tolist()
    x = np.arange(len(labels))
    width = 0.35
    
    plt.figure(figsize=(16, 8))
    plt.bar(x - width/2, df_plot['NB'], width, label='NB (incl. all)', color='skyblue')
    plt.bar(x + width/2, df_plot[new_col_name], width, label=f"NB (excl. {nutrient_to_exclude})", color='deeppink')
    plt.xlabel('Food Item')
    plt.ylabel('NB Value (%)')
    plt.title(f"Comparison of NB with and without {nutrient_to_exclude}")
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    df_plot['NB_diff'] = df_plot['NB'] - df_plot[new_col_name]
    df_plot['NB_pct_change'] = (df_plot['NB_diff'] / df_plot['NB']) * 100
    return df_plot


In [17]:
def compute_nb_excluding_multiple(row, nutrient_list, exclude_list, scaling_factor=2000):
    new_list = nutrient_list.copy()  
    for nutr in exclude_list:
        if nutr in new_list:
            new_list.remove(nutr)
    return compute_nb(row, new_list, scaling_factor=scaling_factor)

In [18]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_butter_margarine_spreads, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )

    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_butter_margarine_spreads, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [19]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- folate
- vitamin b1
- RAE
- vitamin b12
- vitamin b6
- vitamin d
- vitamin e

In [20]:
exclude_list = ['vitamin_b1_eaten', 'vitamin_b6_eaten', 'folate_eaten', 'vitamin_b12_eaten', 'vitamin_d_eaten', 'vitamin_e_eaten', 'all_trans_retinol_equivalents_activity_eaten']
qualifying_nutrients = list(nutrient_info.keys())

for idx, row in high_qi_butter_margarine_spreads.iterrows():

    qi_incl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=[])
    qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    nb_incl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=[]) 
    nb_excl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    print(f"Food item : {row['combined_name']}")
    print(f"QI (incl. all): {qi_incl}")
    print(f"QI (excl. {', '.join(exclude_list)}): {qi_excl}")
    print(f"NB (incl. all): {nb_incl:.2f}")
    print(f"NB (excl. {', '.join(exclude_list)}): {nb_excl:.2f}")
    print("-----------")

Food item : Délice : extra Light
QI (incl. all): 3.154637880388236
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 1.0525056592464295
NB (incl. all): 53.24
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 39.22
-----------
Food item : le Parfait Wurziger Brotaufstrich
QI (incl. all): 3.0103077877220494
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 1.892901392001471
NB (incl. all): 74.61
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 74.73
-----------
Food item : Becel: Pro Activ
QI (incl. all): 2.2088835749677593
QI (excl. vitamin_b1_eaten, vit

In [21]:
df_butter_margarine_spreads[df_butter_margarine_spreads['combined_name'] == 'Cenovis, yeast spread'].drop_duplicates('combined_name')[[i for i in df_butter_margarine_spreads if '_ratio_scaled' in i]].T

,247590
folate_eaten_ratio_scaled,0.000000
niacin_eaten_ratio_scaled,0.000000
pantothenic_acid_eaten_ratio_scaled,0.000000
vitamin_b2_eaten_ratio_scaled,0.000000
vitamin_b1_eaten_ratio_scaled,0.000000
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.000000
vitamin_b12_eaten_ratio_scaled,0.000000
vitamin_b6_eaten_ratio_scaled,0.000000
vitamin_c_eaten_ratio_scaled,0.000000
